In [2]:
import torch
import torch.nn as nn
import math

# Each gate is a class

In [3]:

class Gate():
    """
    Implements gates for transient rewiring of a network. This is inspired by the paper:
    
    Nikolić, D. (2023). Where is the mind within the brain? Transient selection of subnetworks 
    by metabotropic receptors and G protein-gated ion channels. 
    Computational Biology and Chemistry, 103, 107820.

    Note: currently works only for 1D layers of neurons
    
    x, y: determine which connection is being gated by this gate, where x and y are indexes of neuron in
        input and output layer for that weight, respectively.
        x can have the value "bias" in case the gate gates the bias parameter.
        
    trigger_layer: in which layer the trigger for this gate is located, input or output. The values
            are "down" for input and "up" for output.
            
    trigger_neuron_id: the neuron index that will provide inputs for triggering this gate
    
    g_activated: the gating value g that will be set once the gate is activated. 
    
    activation_threshold: The minimal output value of the trigger neuron necessary to activate the gates. 
    The default value is 0.1. 
    
    g_default: The value of the gating parameter in its inactivatged state. The deafult value is 1.0.
    
    duration: deterimines the number of iteratations after which the gate will return back 
        to its default state. The default number of iterations is 5.
    
    Usage:
    gate = Gate(g, neuron_id)
    
    
    We have the following methods:
    
    
    get_trigger_neuron(): returns the trigger neuron index and the layer (up or down)
    
    sniff(): check the input and decides whether to activate the gate; it does nothing if the gates is
            already active.
            
    """

    def __init__(self, x, y, trigger_layer, trigger_neuron_id, g_activated, activation_threshold = 0.1, g_default = 1.0, duration = 5):

        if x == "bias":
            self.n_x = -1
        else:
            self.n_x = x
        self.n_y = y
        if trigger_layer == "down":
            self.layer = trigger_layer #up or down
        else:
            self.layer = "up"
        self.g_activated = g_activated
        self.neuron_id = trigger_neuron_id
        self.activation_threshold = activation_threshold
        self.g_default = g_default
        self.duration = duration
        self.state = self.g_default
        self.counter = 0
        self.activated = False

    def get_trigger_neuron(self):
        return self.layer, self.neuron_id

    def sniff(self, input):
        if self.counter == 0 and input > self.activation_threshold:
                self.state = self.g_activated
                self.counter = self.duration
                self.activated = True
        self.counter_()

    def counter_(self):
        if self.counter > 0:
            self.counter -= 1
        else:
            self.state = self.g_default
            self.activated =  False
            
    def de_activate(self):
        self.activated = False
        self.counter = 0
        self.state = self.g_default
    

# Here we expand the functionality of a PyTorch layer to accommodate gates

In [4]:

class GatedLinear(nn.Module):
    """
    This overwrites the Linear function of the nn module. The new forward function also applies gates
    
    The constructor receivies a list of gates to operate within this layer
    
    forward() is extended so that it respects the gate values
    
    think(): this is a function that updates gates of that layer
    """
    def __init__(self, in_features, out_features, set_gates=None, bias=True):
        super(GatedLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        if set_gates is not None:
            self.gating = True
            self.g_weight = nn.Parameter(torch.ones(out_features, in_features))
            self.g_weight.requires_grad=False
            if bias:
                self.g_bias = nn.Parameter(torch.ones(out_features))
                self.g_bias.requires_grad=False
            else:
                self.register_parameter('g_bias', None)
            
        else:
            self.gating = False
            self.register_parameter('g_weight', None)
            self.register_parameter('g_bias', None)
            
        self.reset_parameters()        
        if set_gates is not None:        
            self.gates = set_gates()      

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input, use_gates = True):            
        if self.gating is True and use_gates is True:
            linear_output = torch.matmul(input, self.weight.t() * self.g_weight.t()) 
        else:
            linear_output = torch.matmul(input, self.weight.t())
            
        if self.bias is not None:
            if self.gating is True and use_gates is True:
                linear_output += self.g_bias * self.bias
            else:
                linear_output += self.bias
            
        return linear_output

    def think(self, output_down, output_up):
        self.g_weight.fill_(1.0)
        self.g_bias.fill_(1.0)
        
        for g in self.gates:
            layer, n_id = g.get_trigger_neuron()
            if layer == "down":
                g.sniff(output_down[n_id])
            else:
                g.sniff(output_up[n_id])
            
            if g.n_x >= 0:
                self.g_weight[g.n_x, g.n_y] *= g.state
            else:
                self.g_bias[g.n_y] *= g.state
        return self.gates
    
    def de_activate_all_gates(self):
        for g in self.gates:
            g.de_activate()
    

# Creating a neural network layer with gates

In [5]:

# Define gates with different activation behaviors
def example_gates():
    g1 = Gate(0, 1, "down", trigger_neuron_id=0, g_activated=10, activation_threshold=0.5)
    g2 = Gate(1, 0, "down", trigger_neuron_id=1, g_activated=5, activation_threshold=0.3)
    g3 = Gate('bias', 1, "down", trigger_neuron_id=0, g_activated=0.2, activation_threshold=0.6)
    return [g1, g2, g3]

# Instantiate the GatedLinear layer with example gates
gl = GatedLinear(2, 2, set_gates=example_gates)

# Manually set the weights and biases to observe the effect of gating
gl.weight = nn.Parameter(torch.tensor([[0.8, 0.2],
                                       [0.3, 0.7]]))
gl.bias = nn.Parameter(torch.tensor([0.1, -0.2]))

# Define a sequence of inputs to test gating effects

In [6]:
inputs = [
    torch.tensor([0.2, 0.2]),  # Below threshold, should not activate gates
    torch.tensor([0.6, 0.2]),  # Activates g1
    torch.tensor([0.3, 0.4]),  # Activates g2
    torch.tensor([0.7, 0.6])   # Activates multiple gates
]    

# Compare to the outputs without using gates

In [7]:

print("=== Outputs with Gates Enabled ===")
for i, inp in enumerate(inputs, 1):
    out = gl.forward(inp, use_gates=True)
    gl.think(inp, out)  # Update gates based on input and output
    print(f"Input {i}: {inp.numpy()}, Output: {out.detach().numpy()}, Gate States: {[g.state for g in gl.gates]}")

# Reset gate states for comparison
gl.de_activate_all_gates()

print("\n=== Outputs with Gates Disabled ===")
for i, inp in enumerate(inputs, 1):
    out = gl.forward(inp, use_gates=False)
    print(f"Input {i}: {inp.numpy()}, Output: {out.detach().numpy()}")

=== Outputs with Gates Enabled ===
Input 1: [0.2 0.2], Output: [0.3 0. ], Gate States: [1.0, 1.0, 1.0]
Input 2: [0.6 0.2], Output: [0.62000006 0.11999999], Gate States: [10, 1.0, 1.0]
Input 3: [0.3 0.4], Output: [1.14 0.17], Gate States: [10, 5, 1.0]
Input 4: [0.7 0.6], Output: [1.86      1.2699999], Gate States: [10, 5, 0.2]

=== Outputs with Gates Disabled ===
Input 1: [0.2 0.2], Output: [0.3 0. ]
Input 2: [0.6 0.2], Output: [0.62000006 0.11999999]
Input 3: [0.3 0.4], Output: [0.42000002 0.17      ]
Input 4: [0.7 0.6], Output: [0.78000003 0.43      ]


# Explanation of Outputs

In this example, we see how dynamic gating influences the network's output. Each input triggers different gate states, which alter the network's behavior by amplifying or reducing specific connections.

#### Outputs with Gates Enabled
With gates enabled, the outputs vary based on each input's ability to trigger specific gates. Here’s a breakdown:

- **Input 1**: `[0.2, 0.2]`
  - **Gate States**: `[1.0, 1.0, 1.0]`
    - All gates remain in their default states (1.0), as the inputs are below the activation thresholds.
  - **Output**: `[0.3, 0.0]`
    - Since the gates haven’t activated, the output is based on the initial weight and bias values without amplification.

- **Input 2**: `[0.6, 0.2]`
  - **Gate States**: `[10, 1.0, 1.0]`
    - `g1` activates because the input `[0.6]` exceeds its threshold of `0.5`, increasing its gating factor to `10`.
  - **Output**: `[0.62000006, 0.11999999]`
    - The amplified gate factor for `g1` increases the contribution of `Input1` to `Hidden2`, leading to a higher output.

- **Input 3**: `[0.3, 0.4]`
  - **Gate States**: `[10, 5, 1.0]`
    - `g2` activates (since `0.4` exceeds its threshold of `0.3`), while `g1` remains active from the previous input.
  - **Output**: `[1.14, 0.17]`
    - Both `g1` and `g2` are amplifying their respective connections, further increasing the output.

- **Input 4**: `[0.7, 0.6]`
  - **Gate States**: `[10, 5, 0.2]`
    - `g3` activates because `0.7` exceeds its threshold, reducing its gating factor to `0.2`, while `g1` and `g2` remain active.
  - **Output**: `[1.86, 1.2699999]`
    - With `g1` and `g2` amplifying their connections and `g3` reducing the bias’s effect, we see a unique output.

#### Outputs with Gates Disabled
In this section, the outputs are calculated without any gate effects (`use_gates=False`). This comparison highlights the impact of gating:

- **Input 1 to Input 4**:
  - Outputs are notably lower because the gate factors are set to their defaults (`1.0`), meaning no amplification or reduction is applied.
  - This contrast shows how gating dynamically controls connection strengths, enabling unique responses based on the input pattern.

### Summary
This example demonstrates how gating can selectively amplify or reduce specific connections in the network. Activated gates dynamically adjust outputs by modifying the contribution of individual inputs, allowing the network to adapt its response based on input conditions.
